# Decision Tree Algorithm
- Implementation of Decision Tree Algorithm on Iris DataSet

### Import Statements

In [1]:
import numpy as np
import pandas as pd
import math as m
from sklearn import datasets
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.metrics import accuracy_score

### Loading Iris Dataset

In [2]:
Iris = datasets.load_iris()
X = Iris.data
Y = Iris.target

X_train, X_test, Y_train , Y_test  = train_test_split(X, Y)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_train.shape)


(112, 4)
(38, 4)
(112,)
(112,)


# Decision Tree Class

In [7]:
class DecisionTree:
    def __init__(self, level = 0):
        self.level = 0
        self.featureName = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'] 
    
    #####  transform_data : Merging Xtrain and Ytrain to form data
    def transform_data(self, X_train , Y_train):
        Y_train = Y_train.reshape(len(Y_train),1)
        data = np.concatenate((X_train,Y_train), axis=1)
        return data
    
    ##### Check Purity  - To check if node is leaf node or not
    def check_purity(self, data):
        label_column = [int(i) for i in data[:,-1]]
        unique_classes = np.unique(label_column)
        if(len(unique_classes) == 1):
            return True
        else:
            return False
        
    ##### Classify data - It classifies which class does node belong if node is leaf node    
    def classify_data(self, data):
        label_column = [int(i) for i in data[:,-1]]
        unique_classes, count_unique_classes = np.unique(label_column, return_counts=True)
        classification = unique_classes[count_unique_classes.argmax()]
        return classification
   
    ##### Get Potential Splits - It returns dictionary that contains columns as keys and average potential sorted values as values of dictionary
    def get_potential_split(self, data):
        potential_splits = {}
        n_columns = len(data[0])-1
        for c in range(n_columns):
            potential_splits[c] = []
            values = data[:, c]
            unique_values = np.unique(values)

            for i in range(1,len(unique_values)):
                ps = (unique_values[i] + unique_values[i-1]) / 2
                potential_splits[c].append(ps)
        return potential_splits
    
    # split_data : to split data into two parts i.e. data_left and data_right by some criteria i.e. split_value and split_column
    def split_data(self, data, split_column, split_value):
        split_column_values = data[:, split_column]

        data_left = data[split_column_values <= split_value]
        data_right = data[split_column_values > split_value]

        return data_left, data_right
    
    ##### Entropy, Infomation Required , Split Information
          # - These helper function calculate and return entropy , information required , split information which is used to calculate gain ratio of particular node
          # - Entropy = -Summation(p(i)*log2(p(i)))
    def entropy(self, data):
        labels_values = [int(i) for i in data[:,-1]]
        _, counts = np.unique(labels_values, return_counts=True)
        probabilities = counts / counts.sum()
        entropies = sum(probabilities * -np.log2(probabilities))
        return entropies

    def info_req(self, data_left, data_right):
        n_total = len(data_left) + len(data_right)
        p_left = len(data_left) / n_total
        p_right = len(data_right) / n_total
        return (p_left * self.entropy(data_left) + p_right* self.entropy(data_right))

    def split_info(self, data_left, data_right):
        n_total= len(data_left) + len(data_right)
        return (- (len(data_left) / n_total) * np.log(len(data_left) / n_total) - (len(data_right) / n_total) * np.log(len(data_right) / n_total))
    
    ##### Gain Ratio - Returns gain ratio for particular node
    def gain_ratio(self, data, data_left, data_right):
        si = self.split_info(data_left, data_right)
        ig = self.entropy(data) - self.info_req(data_left, data_right)
        return (ig / si)
    
    ##### Find Best Split Helper Function - This function takes Xtrain data and potential splits dictionary and 
    #                                                        returns the best split column and best split value
    def find_best_split(self, data, potential_splits):
        max_entropy = m.inf
        max_gain = -m.inf
        for c in potential_splits:
            for v in potential_splits[c]:
                data_left, data_right = self.split_data(data, split_column=c, split_value=v)
                current_entropy = self.info_req(data_left, data_right)
                current_gain = self.gain_ratio(data, data_left , data_right)


                if current_entropy < max_entropy:
                    max_gain = current_gain
                    max_entropy = current_entropy
                    best_split_column = c
                    best_split_value = v
        return best_split_column, best_split_value 

    '''
    data = Xtrain + Ytrain
    f = features
    level = current level
    
    decision_tree function is being called recursively to return binary sub_tree
    '''
    def decision_tree(self, data, f, level):

        if self.check_purity(data) :
            labels_values = [int(i) for i in data[:,-1]]
            _, counts = np.unique(labels_values, return_counts=True)
            print('Level', level)
            for i in range(len(counts)):
                print('Count of', i, '= ', counts[i])
                print('Current Entropy  is = ', self.entropy(data))
                print('Reached leaf Node')
            return self.classify_data(data)


        elif len(f) == 0:
            labels_values = [int(i) for i in data[:,-1]]
            _, counts = np.unique(labels_values, return_counts=True)
            print('Level', level)
            for i in range(len(counts)):
                print('Count of', i, '= ', counts[i])
                print('Current Entropy  is = ', self.entropy(data))
                print('Reached leaf Node')
            return self.classify_data(data)


        else:
            potential_split = self.get_potential_split(data)
            best_split_feature, best_split_value = self.find_best_split(data, potential_split)
            data_left, data_right = self.split_data(data, best_split_feature, best_split_value)

            labels_values = [int(i) for i in data[:,-1]]
            _, counts = np.unique(labels_values, return_counts=True)
            print('Level', level)
            for i in range(len(counts)):
                print('Count of', i, '= ', counts[i])
                print('Current Entropy  is = ', self.entropy(data))
                print('Splitting on feature', f[best_split_feature], 'with gain ratio', self.gain_ratio(data, data_left, data_right))

            data_left = np.delete(data_left, best_split_feature , 1)
            data_right = np.delete(data_right, best_split_feature , 1)

            que = '{} <= {}'.format(f[best_split_feature], best_split_value)
            sub_tree = {que : []}

            f.remove(f[best_split_feature])
            y = self.decision_tree(data_left, f, level+1)
            n = self.decision_tree(data_right, f, level+1)

            sub_tree[que].append(y)
            sub_tree[que].append(n)
            return sub_tree

    ### Fit Function - fits training dataset 
    #                - Returns dictionary generated from decision tree
    def fit(self, Xtrain , Ytrain):
        self.data = self.transform_data(Xtrain , Ytrain)
        self.dictionary = self.decision_tree(self.data, self.featureName, self.level)               # here data is X_Y_train
    
    '''
    predict_row() is helper function which takes one row of testing dataset and dictionary.
    It returns class for particular testing point / row.
    '''

    def predict_row(self, x, dictionary):
    
        col = {'petal width (cm)':3, 'sepal length (cm)':0 , 'sepal width (cm)':1 , 'petal length (cm)' :2}

        q = list(dictionary.keys())[0]
        feature1,feature2,feature3, operator, value = q.split()
        feature = feature1+" "+feature2+" "+feature3

        if x[col[feature]] <= float(value):
            cls = dictionary[q][0]                 
        else:
            cls = dictionary[q][1]


        if isinstance(cls, dict):
            rt = cls
            return self.predict_row(x, rt)                     # recursive calling 
        else:
            return cls
    ### Predict Function - predicts testing dataset
      #                 - Returns y_predicted list    
    def predict(self, xtest):
        output = np.zeros(len(xtest), dtype = int)
        for i in range(len(xtest)):
            row = xtest[i , :]
            output[i] = self.predict_row(row, self.dictionary)
        return output                                          # returning list of predicted classes


In [8]:
clf = DecisionTree()

In [9]:
clf.fit(X_train, Y_train)
ypred_cls = clf.predict(X_test)

Level 0
Count of 0 =  37
Current Entropy  is =  1.5848478277058313
Splitting on feature petal length (cm) with gain ratio 1.442695040888963
Count of 1 =  37
Current Entropy  is =  1.5848478277058313
Splitting on feature petal length (cm) with gain ratio 1.442695040888963
Count of 2 =  38
Current Entropy  is =  1.5848478277058313
Splitting on feature petal length (cm) with gain ratio 1.442695040888963
Level 1
Count of 0 =  37
Current Entropy  is =  0.0
Reached leaf Node
Level 1
Count of 0 =  37
Current Entropy  is =  0.999871756640849
Splitting on feature petal width (cm) with gain ratio 0.9587053216903549
Count of 1 =  38
Current Entropy  is =  0.999871756640849
Splitting on feature petal width (cm) with gain ratio 0.9587053216903549
Level 2
Count of 0 =  36
Current Entropy  is =  0.4689955935892812
Splitting on feature sepal length (cm) with gain ratio 0.7487424363630022
Count of 1 =  4
Current Entropy  is =  0.4689955935892812
Splitting on feature sepal length (cm) with gain ratio 0.

In [11]:
clf.dictionary

{'petal length (cm) <= 2.45': [0,
  {'petal width (cm) <= 1.75': [{'sepal length (cm) <= 4.95': [2,
      {'sepal width (cm) <= 2.55': [1, 1]}]},
    2]}]}

### Checking Accuracy

In [6]:
accuracy_score(Y_test, ypred_cls)

0.9473684210526315